In [ ]:
import os
from os.path import join
import sys 
import glob
import numpy as np
import nibabel as nib
from IPython.display import Image
from nipype.interfaces import fsl
from nipype.interfaces.fsl import utils
from nipype.testing import example_data
from subprocess import call

from nipype.caching.memory import PipeFunc

from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Uncomment for downloading data
# call("hcpdownload.sh")

In [ ]:
subject_id = '100307'

In [ ]:
base_path = '/home/ubuntu/IAAN-LAB/data/hcp/'
main_path = join(base_path, subject_id)

In [ ]:
fmri_path = join(main_path, 'rfMRI_REST1_LR/')
slice_path = join(fmri_path, 'split/vol0000.nii.gz')
fmri_data_path = join(fmri_path, 'rfMRI_REST1_LR.nii.gz')
t1_path =  join(join(main_path, 'T1w'), 'T1w_acpc_dc_restore_brain.nii.gz')
atlas_path = join(main_path, 'aparc+aseg.nii.gz')

In [ ]:
def run_flirt(reference_path, template_path, name):
    flt = fsl.FLIRT(bins= 256, cost_func='corratio')
    flt.inputs.in_file = template_path
    flt.inputs.reference = reference_path
    flt.inputs.output_type = "NIFTI_GZ"
    flt.inputs.out_file = join(main_path, f'{name}.nii.gz')
    flt.inputs.out_matrix_file = join(main_path, f'{name}.mat')
    print('The following command is being run:')
    print(flt.cmdline) # doctest: +ELLIPSIS
    #'flirt -in structural.nii -ref mni.nii -out structural_flirt.nii.gz -omat structural_flirt.mat -bins 640 -searchcost mutualinfo'
    res = flt.run() #doctest: +SKIP

In [ ]:
# load the flirt affine matrix as a text, because flirt saves it that way?
def load_affine(transform_name): 
    aff = open(join(main_path, f'{transform_name}.mat'))
    aff = aff.read()
    aff = [x.split(' ')[:-1] for x in aff.split('\n')[:-1]]
    aff = np.array([[float(y) for y in x[::2]] for x in aff])
    print(f'{transform_name} transformation')
    print(aff)
    return aff

In [ ]:
# load the atlas image, return its data and affine matrix
def load_atlas(): 
    atlas = nib.load(atlas_path)
    print('Shape of the atlas file:', atlas.shape)
    print('Affine matrix for parcellation file', atlas.affine)
    labels = atlas.get_fdata()
    print('Total number of different labels in parcellation file:', len(np.unique(labels)))
    return labels, atlas.affine

In [ ]:
# warp all the fmri slices to atlas / T1 coordinates
# it is slower than warping atlas to 1 fmri slice
# but warping label info and not intensity requires warping, thresholding and merging masks
# we are not dealing with potential errors (e.g. additional/overlapping ROIs) coming from that
def fmri_apply_warp(in_path, out_path, transform_name): 
    masks_warped_path = join(main_path, "masks_warped")
    try:
        os.mkdir(out_path)
    except FileExistsError:
        pass
    
    for file in os.listdir(in_path):
        aw = fsl.ApplyWarp()
        aw.inputs.in_file = join(in_path, file)
        aw.inputs.ref_file = t1_path
        aw.inputs.premat = join(main_path, f'{transform_name}.mat')
        aw.inputs.out_file = join(out_path, file)
        #Note: this command says that the output file shall exist already
        #aw.inputs.out_file = join(path, 'T1w_to_diff.nii.gz')
        
        print(f'Executing the following command for file {file}')
        print(aw.cmdline)
        res = aw.run() #doctest: +SKIP

In [ ]:
# merge warped fmri slices into 1 image with fsl.Merge using PipeFunc
def merge_fmri(path):
    slice_0 = nib.load(join(path, 'vol0000.nii.gz'))
    slice_0_affine = slice_0.affine
    
    path_list = [join(path, 'vol0000.nii.gz'),]
    for i in range(1, 1200):
        num = (4 - len(str(i))) * '0' + str(i)
        filepath = join(path, f'vol{num}.nii.gz')
        path_list.append(filepath)
               
    fsl_merge = PipeFunc(fsl.Merge, base_dir='.')
    out = fsl_merge(in_files=path_list, dimension='t')
    out_path = out.outputs.merged_file    
    return out_path

In [ ]:
transform_name = 'fmri_to_T1w'

In [ ]:
# get the affine transform between fmri and T1 / atlas
run_flirt(t1_path, slice_path, transform_name)

In [ ]:
# load that affine transform
affine_transform = load_affine(transform_name)

In [ ]:
# warp fmri slices
fmri_in_path = join(fmri_path, 'split')
fmri_out_path = join(fmri_path, 'split_warped')
fmri_apply_warp(fmri_in_path, fmri_out_path, transform_name)

In [ ]:
# merge warped fmri slices
merged_fmri = merge_fmri(join(fmri_path, 'split'))

In [ ]:
# fsl.Merge returns filepath in some temp folder - load the merged fmri data from there
merged_fmri_data = nib.load(merged_fmri).get_fdata()

In [ ]:
fmri_at_atlas_path = join(main_path, 'fmri_at_atlas.nii.gz')

In [ ]:
# re-save the merged fmri in fmri_at_atlas_path
slice_0 = nib.load(join(fmri_path, 'split/vol0000.nii.gz'))
slice_0_affine = slice_0.affine 
img = nib.Nifti1Image(merged_fmri_data, slice_0_affine)
img.to_filename(fmri_at_atlas_path)

In [ ]:
# confirm that the shape of the new fmri data is the same
# should have 1200 (number of frames) as the last, 4th dimension
merged_fmri_data.shape

In [ ]:
# load the atlas
labels, atlas_affine_mat = load_atlas()

In [ ]:
# check how many labels are there in atlas
# if it is N, connectivity matrix should be NxN 
len(np.unique(labels))

Connectivity matrix. Nilearn library operates with file paths, not only get_fdata-loaded objects.

In [ ]:
connectome_measure = ConnectivityMeasure(kind='correlation')

In [ ]:
masker = NiftiLabelsMasker(labels_img=atlas_path, standardize=True,
                           memory='nilearn_cache')

In [ ]:
masker_fit_transform = masker.fit_transform(fmri_at_atlas_path)

In [ ]:
masker_fit_transform.shape

In [ ]:
# produce correlation matrix
# this function works on a list of subjects, so passing a 1-item list
correlation_matrices = connectome_measure.fit_transform([masker_fit_transform,])

In [ ]:
# by the same logic, getting the matrix for the first and only subject
correlation_matrix = correlation_matrices[0]

In [ ]:
# confirm the matrix shape
correlation_matrix.shape

In [ ]:
# save the matrix
matrix_path = join(main_path, 'correlation_matrix.txt')
np.savetxt(matrix_path, correlation_matrix)

In [ ]:
# mask for not showing diagonal elements
diagonal_mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

In [ ]:
# plot the matrix
f, ax = plt.subplots(figsize=(11, 9))
# cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(correlation_matrix, mask=diagonal_mask, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})